In [1]:
import tensor_comprehensions as tc
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [ ]:
# as recommended by the authors for better performance
tune_settings = {
    "threads": 32,
    "generations": 7,
    "pop_size": 15,
    "number_elites": 3
}

In [2]:
# heavier options
tune_settings = {
    "threads": 32,
    "generations": 20,
    "pop_size": 50,
    "number_elites": 3
}

In [3]:
conv_lang = tc.database['convolution_strided']['lang']

In [15]:
# try some different sizes

I = torch.randn(512, 3, 20, 20).cuda()
W1 = torch.randn(6, 3, 3, 3).cuda()
B = torch.randn(6).cuda()

In [16]:
conv = tc.define(conv_lang.format(sh=3, sw=3), name='convolution_strided')

In [21]:
conv = tc.define(conv_lang.format(sh=3, sw=3), name='convolution_strided')
conv.autotune(I, W1, B,
              cache='cache/large_batch.tc',
              **tune_settings,
              options=tc.Options('conv'))

[INFO]: Autotuning cache will be saved to: cache/large_batch.tc.cuda/options


In [22]:
# run twice to prep cuda
out = conv(I, W1, B, cache='cache/large_batch.tc')
torch.cuda.synchronize()
print(out.size())


torch.Size([512, 6, 6, 6])


In [35]:
# test performance using TC
output = torch.zeros(512, 6, 6, 6).cuda()
torch.cuda.synchronize()

timings = np.zeros(1000)

for i in range(1000):
    start = time.perf_counter()
    output = conv(I, W1, B, cache='cache/large_batch.tc')
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)


total time: 0.104475689005
[  6.14650000e-05   8.17290002e-05   1.49113200e-04]


In [34]:
# test the torch equivalent

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 6, (3, 3), stride=(3, 3)))
    
    def forward(self, x):
        return self.layer(x)

CN = ConvNet()
CN.cuda()

var_input = Variable(torch.randn(512, 3, 20, 20).cuda())
# run a couple times to work out start hiccups
CN(var_input)
torch.cuda.synchronize()
CN(var_input)
torch.cuda.synchronize()


# test performance
timings = np.zeros(100)

for i in range(100):
    start = time.perf_counter()
    output = CN(var_input)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)

total time: 0.00830888600171
[  7.16190000e-05   7.70214999e-05   1.04412000e-04]
